In [1]:
# Import modules and packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import tensorflow as tf

from datetime import datetime
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.losses import Huber, MeanSquaredError
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

from functools import reduce

%matplotlib inline

In [2]:
def retrieve_datasets():
# Importing Training Set
    path=os.path.abspath(os.path.join(".", os.pardir))
    
    eia_df = pd.read_csv(path+'/data/all_eia_stock_sheet_latest.csv',header=0,
                                infer_datetime_format=True, delimiter=';',parse_dates=['Date'], index_col=['Date']) 	#Date 2	Aug 27, 1982
    
    eia_pricing_df = pd.read_csv(path+'/data/eia_pricing_latest.csv',header=0,
                                infer_datetime_format=True, delimiter=';',parse_dates=['Date'], index_col=['Date']) 	#Date 2	Aug 27, 1982
    #eia_pricing_df = eia_pricing_df.shift(0)
    
    icsa_df = pd.read_csv(path+'/data/ICSA_current.csv',header=0,
                          infer_datetime_format=True, delimiter=',',
                          parse_dates=['DATE'] ,index_col=['DATE'])
   
    #reduce scale to per barrel pricing
    eia_df = eia_df.div(10000).round(5)
    icsa_df = icsa_df.div(10000).round(5)


    #shift back one day to match and pair with s&p dates that line up with EIA (all_barrells_df) dates
    icsa_df.index = icsa_df.index + pd.Timedelta(days=-1)
    #rename index and shift back 4 periods as unemployement data *should* be indicator later...periods = weeks
    icsa_df.index = icsa_df.index.rename('Date')
    icsa_df = icsa_df.shift(periods=4)
      
    return (eia_df, eia_pricing_df, icsa_df)


def combine_datasets(datasets):
    """
    We expect an list of dataframes to iterate over and create a final combined df
    Expected column key for the join is 'Date'
    """
    df_list = list(range(0, len(datasets)))
    dataset_list = list()
    for i in df_list:
        dataset_list.append(datasets[i])
       
    combo_df = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                                   how='inner'), dataset_list)
    combo_df = combo_df.sort_index(ascending=True)


    combo_df = combo_df[['WCESTUS1', 'WCESTP21', 'WCRSTUS1', 'WCESTP31', 'WCSSTUS1','WTTSTUS1','ICSA','RCLC1']] #'WCSSTUS1',
    combo_df.dropna(inplace=True) #just in case, drop non numbers
    
    #only use data from 2010 as this is beginning of some of the columns of data in eia barrels data
    combo_df = combo_df['2010-01-01':'2021-01-01']

    return combo_df


datasets = retrieve_datasets()
combo_df = combine_datasets(datasets)

dataset_train = combo_df

#columns to be used in training/prediction
cols = ['WCESTUS1', 'WCESTP21', 'WCRSTUS1', 'WCESTP31', 'WCSSTUS1','WTTSTUS1','ICSA','RCLC1']

cols = list(cols)

# Extract dates (will be used in visualization)
datelist_train = list(dataset_train.index)

print(f'Training set shape: {dataset_train.shape}')
print(f'All timestamps:    {len(datelist_train)}')
len(dataset_train)

Training set shape: (567, 8)
All timestamps:    567


567

In [3]:
#shift for training/validation
#train_len = int(len(dataset_train*.8)
#val_len = len(len(dataset_train)) - train_len

#X_val = X_train[train_len:]
#y_val = y_train[train_len:]
#X_train = X_train[:train_len]
#y_train = y_train[:train_len]


In [4]:

dataset_train = dataset_train.astype(float)

# Using multiple features (predictors)
training_set = dataset_train.values

print('Shape of training set == {}.'.format(training_set.shape))
training_set

Shape of training set == (567, 8).


array([[ 30.8283,   8.1858, 103.4898, ..., 175.1934,  45.6   ,  74.03  ],
       [ 31.0577,   8.0047, 103.7189, ..., 175.3694,  46.9   ,  74.59  ],
       [ 31.3662,   7.9276, 104.0274, ..., 175.1277,  50.7   ,  73.91  ],
       ...,
       [ 48.8721,  14.6251, 112.6911, ..., 199.2034,  75.8   ,  41.7   ],
       [ 48.8042,  14.6859, 112.6232, ..., 199.215 ,  75.7   ,  44.8   ],
       [ 50.3231,  14.674 , 114.1316, ..., 201.1971,  71.1   ,  45.41  ]])

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

np.info(training_set_scaled)


class:  ndarray
shape:  (567, 8)
strides:  (8, 4536)
itemsize:  8
aligned:  True
contiguous:  False
fortran:  True
data pointer: 0x6737290
byteorder:  little
byteswap:  False
type: float64


In [6]:
# Creating a data structure with 90 timestamps and 1 output
X_train = []
y_train = []

n_future = 2   # Number of weeks we want to predict into the future
n_past = 4    # Number of past weeks we want to use to predict the future

for i in range(n_past, len(training_set_scaled) - n_future +1):
    X_train.append(training_set_scaled[i - n_past:i, 0:dataset_train.shape[1]])
    y_train.append(training_set_scaled[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

train_len = int(len(X_train)*.8)
val_len = len(X_train) - train_len

#shift for training/validation
"""
X_val = X_train[train_len:]
y_val = y_train[train_len:]
X_train = X_train[:train_len]
y_train = y_train[:train_len]

#X_val, y_val = 

print(f'X_train shape == {X_train.shape}')
print(f'y_train shape == {y_train.shape}')

print(len(X_train))
print(len(X_val))
"""

"\nX_val = X_train[train_len:]\ny_val = y_train[train_len:]\nX_train = X_train[:train_len]\ny_train = y_train[:train_len]\n\n#X_val, y_val = \n\nprint(f'X_train shape == {X_train.shape}')\nprint(f'y_train shape == {y_train.shape}')\n\nprint(len(X_train))\nprint(len(X_val))\n"

<h4>Create a model & Training</h4>

In [7]:
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_past, dataset_train.shape[1])))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(units=256, return_sequences=False))
model.add(Dropout(0.25))
model.add(Dense(units=1, activation='relu'))

# Compiling the Neural Network
#model.compile(optimizer = Adam(learning_rate=0.001), loss=Huber())

<h3>Start training</h3>

In [8]:
%%time
MAX_EPOCHS = 100
#es = EarlyStopping(monitor='val_loss', mode='auto',min_delta=1e-7, patience=10, verbose=0)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=0)
mcp = ModelCheckpoint(filepath='models/mv_latest_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True)
tb = TensorBoard('logs')
#callbacks=[es, rlr, mcp, tb]
#history = model.fit(X_train, y_train, shuffle=True, epochs=25, validation_split=0.05, verbose=1, batch_size=32,callbacks=[es, rlr, mcp, tb]) 

#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,mode='min')


early_stopping = EarlyStopping(monitor='loss', patience=10, mode='auto', min_delta=1e-5)

model.compile(optimizer = Adam(learning_rate=0.001), loss=Huber())
history = model.fit(X_train, y_train, shuffle=True, batch_size=16, epochs=MAX_EPOCHS,callbacks=[early_stopping])
#validation_data=(X_val, y_val)


SyntaxError: invalid syntax (<unknown>, line 16)

<p>
Notes:<br>
<ul>
<li><b>EarlyStopping</b> - Stop training when a monitored metric has stopped improving.</li>
<li><code>monitor</code> - quantity to be monitored.</li>
<li><code>min_delta</code> - minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than <code>min_delta</code>, will count as no improvement.</li>
<li><code>patience</code> - number of epochs with no improvement after which training will be stopped.</li>
</ul>

<ul>
<li><b>ReduceLROnPlateau</b> - Reduce learning rate when a metric has stopped improving.</li>
<li><code>factor</code> - factor by which the learning rate will be reduced. <code>new_lr = lr * factor</code>.</li>
</ul>
</p>

<hr>

<p>
The last date for our training set is <code>30-Dec-2016</code>.<br>
</p>

<p>
We will perform predictions for the next <b>20</b> days, since <b>2017-01-01</b> to <b>2017-01-20</b>.
</p>

<p>
Notes:<br>
<ul>
<li><b>EarlyStopping</b> - Stop training when a monitored metric has stopped improving.</li>
<li><code>monitor</code> - quantity to be monitored.</li>
<li><code>min_delta</code> - minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than <code>min_delta</code>, will count as no improvement.</li>
<li><code>patience</code> - number of epochs with no improvement after which training will be stopped.</li>
</ul>

<ul>
<li><b>ReduceLROnPlateau</b> - Reduce learning rate when a metric has stopped improving.</li>
<li><code>factor</code> - factor by which the learning rate will be reduced. <code>new_lr = lr * factor</code>.</li>
</ul>
</p>

<hr>

<p>
The last date for our training set is <code>30-Dec-2016</code>.<br>
</p>

<p>
We will perform predictions for the next <b>20</b> days, since <b>2017-01-01</b> to <b>2017-01-20</b>.
</p>

In [ ]:
# Generate list of sequence of days for predictions
datelist_future = pd.date_range(datelist_train[-1], periods=n_future, freq='7d').tolist()
print((datelist_train[-1]))
'''
Remeber, we have datelist_train from begining.
'''

# Convert Pandas Timestamp to Datetime object (for transformation) --> FUTURE
datelist_future_ = []
for this_timestamp in datelist_future:
    datelist_future_.append(this_timestamp.date())
    
datelist_future

In [ ]:
# Perform predictions
predictions_future = model.predict(X_train[-n_future:])

predictions_train = model.predict(X_train[n_past:])

print(np.info(predictions_future))
print('------------------')
print(np.info(predictions_train))
print('------------------')
print(predictions_future)
print('------------------')
print(predictions_future.reshape(2,1))
predictions_future = predictions_future.reshape(2,1)
predictions_train = predictions_train.reshape(len(predictions_train),1)


In [ ]:
# Inverse the predictions to original measurements
# ---> Special function: convert <datetime.date> to <Timestamp>
def datetime_to_timestamp(x):
    '''
        x : a given datetime value (datetime.date)
    '''
    return datetime.strptime(x.strftime('%Y%m%d'), '%Y%m%d')

y_pred_future = sc_predict.inverse_transform(predictions_future)
y_pred_train = sc_predict.inverse_transform(predictions_train)

PREDICTIONS_FUTURE = pd.DataFrame(y_pred_future, columns=['WCESTUS1']).set_index(pd.Series(datelist_future))
PREDICTION_TRAIN = pd.DataFrame(y_pred_train, columns=['WCESTUS1']).set_index(pd.Series(datelist_train[2 * n_past + n_future -1:]))

# Convert <datetime.date> to <Timestamp> for PREDCITION_TRAIN
PREDICTION_TRAIN.index = PREDICTION_TRAIN.index.to_series().apply(datetime_to_timestamp)

#re-multiply to return to orig sizes
dataset_train.WCESTUS1 = dataset_train.WCESTUS1.multiply(10000).round(0)
PREDICTIONS_FUTURE = PREDICTIONS_FUTURE.multiply(10000).round(0)
PREDICTION_TRAIN['WCESTUS1'] = PREDICTION_TRAIN['WCESTUS1'].multiply(10000).round(0)
PREDICTION_TRAIN,PREDICTIONS_FUTURE

<h3>Step #6. Visualize the Predictions</h3>

In [ ]:
# Set plot size 
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8

# Plot parameters
START_DATE_FOR_PLOTTING = '2019-11-01'

plt.plot(PREDICTIONS_FUTURE.index, PREDICTIONS_FUTURE['WCESTUS1'], color='r', label='Predicted Stock Price')
plt.plot(PREDICTION_TRAIN.loc[START_DATE_FOR_PLOTTING:].index, PREDICTION_TRAIN.loc[START_DATE_FOR_PLOTTING:]['WCESTUS1'], color='orange', label='Training predictions')
plt.plot(dataset_train.loc[START_DATE_FOR_PLOTTING:].index, dataset_train.loc[START_DATE_FOR_PLOTTING:]['WCESTUS1'], color='b', label='Actual Stock Price')

plt.axvline(x = min(PREDICTIONS_FUTURE.index), color='green', linewidth=2, linestyle='--')

plt.grid(which='major', color='#cccccc', alpha=0.2)

plt.legend(shadow=True)
plt.title('Predcitions and Acutal Stock Prices', family='Times', fontsize=12)
plt.xlabel('Timeline', family='Arial', fontsize=10)
plt.ylabel('Stock Price Value', family='Arial', fontsize=10)
plt.xticks(rotation=45, fontsize=8)
plt.show()